In [1]:
import logging
from pathlib import Path
from Pegasus.api import *
import glob
import os
logging.basicConfig(level = logging.DEBUG)

# import os
# os.environ['KAGGLE_USERNAME'] = "vedula"
# os.environ['KAGGLE_KEY'] = "482a5c14ced45f63f3698eacb8fa0c62"

# import kaggle
# kaggle.api.dataset_download_files('nikhilpandey360/chest-xray-masks-and-labels/download', path='.', unzip=True)

In [6]:
pwd

'/home/scitech/shared-data/Mask_RCNN/samples/lung'

In [2]:
preprocess = Transformation(
                "preprocess",
                site="local",
                pfn="/home/scitech/shared-data/Mask_RCNN/samples/lung/preprocess.py",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )


In [3]:
tc = TransformationCatalog().add_transformations(preprocess).write()

In [4]:
file_list = []
output_list = []

rc = ReplicaCatalog()



for f in glob.glob("*.png"):
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./train_images/").resolve() / f)
    
rc.write()

for filename in glob.glob("*.png"):
    f = filename.strip(".png")+"_norm.png"
    output_list.append(File(f))
    
wf = Workflow("preprocess")
    
job_preprocess = Job(preprocess)\
                    .add_inputs(*file_list)\
                    .add_outputs(*output_list)

wf.add_jobs(job_preprocess)


In [5]:
try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)       

Plan:
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.08.04 18:03:28.135 UTC:    
2020.08.04 18:03:28.140 UTC:   ----------------------------------------------------------------------- 
2020.08.04 18:03:28.145 UTC:   File for submitting this DAG to HTCondor           : preprocess-0.dag.condor.sub 
2020.08.04 18:03:28.151 UTC:   Log of DAGMan debugging messages                 : preprocess-0.dag.dagman.out 
2020.08.04 18:03:28.157 UTC:   Log of HTCondor library output                     :

[############################----------------------]  55.6% ..Failure (Completed: 5, Queued: 0, Running: 0, Failed: 1)
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

 Submit Directory   : /home/scitech/shared-data/Mask_RCNN/samples/lung/scitech/pegasus/preprocess/run0006
 Total jobs         :      9 (100.00%)
 # jobs succeeded   :      5 (55.56%)
 # jobs failed      :      1 (11.11%)
 # jobs held        :      0 (0.00%)
 # jobs unsubmitted :      3 (33.33%)

******************************Failed jobs' details******************************

===========================stage_out_local_local_0_0============================

 last state: POST_SCRIPT_FAILED
       site: local
submit file: 00/00/stage_out_local_local_0_0.sub
output file: 00/00/stage_out_local_local_0_0.out.001
 error file: 00/00/stage_out_local_local_0_0.err.001

-------------------------------Task #1 - Summary-----------------------